In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Download the zip file of MLQA


# MLQA

In [ ]:
!wget https://dl.fbaipublicfiles.com/MLQA/MLQA_V1.zip

Extract data for zip file

In [ ]:
import zipfile
with zipfile.ZipFile('/kaggle/working/MLQA_V1.zip') as zip_ref:
    zip_ref.extractall('/kaggle/working/')

Import libraries

In [ ]:
import os
import sys
import random
import argparse
import json
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm

# sys.setdefaultencoding('utf8')
random.seed(42)
np.random.seed(42)

Load the extracted data in train_file and test_file

In [ ]:
mlqa_train_data = '/kaggle/working/MLQA_V1/dev/dev-context-hi-question-hi.json'
mlqa_test_data = '/kaggle/working/MLQA_V1/test/test-context-hi-question-hi.json'

with open(mlqa_train_data, 'r') as file_input:
    train_file = json.load(file_input)
    
with open(mlqa_test_data, 'r') as file_input:
    test_file = json.load(file_input)

Design a function to Process the load data and return it in a dict form

In [ ]:
def preprocess(dataset, tier):
    num_exs = 0 
    examples = []

    for articles_id in tqdm(range(len(dataset['data'])), desc="Preprocessing {}".format(tier)):
        article_paragraphs = dataset['data'][articles_id]['paragraphs']
        for pid in range(len(article_paragraphs)):
            context = article_paragraphs[pid]['context']
            qas = article_paragraphs[pid]['qas'] 
            for qn in qas:
                question = qn['question'] 
                ans_text = qn['answers'][0]['text']
                ans_start_charloc = qn['answers'][0]['answer_start']
                ans_end_charloc = ans_start_charloc + len(ans_text)
                examples.append(
                    {
                        # 'id':articles_id,
                        'context':context, 
                        'question':question, 
                        'answer_text':ans_text, 
                        'answer_start':ans_start_charloc, 
                        # 'answer_end':ans_end_charloc
                    }
                )

                num_exs += 1
    print(num_exs)    
    return examples

Process the train_file and test_file by using abouve process funtion

In [ ]:
examples_train = preprocess(train_file, 'dev')
examples_test = preprocess(test_file, 'test')

Add a colum of language 

In [ ]:
examples = examples_train + examples_test
mlqa = pd.DataFrame(examples)
mlqa['language'] = 'hindi'

# xquad

In [ ]:
!git clone https://github.com/deepmind/xquad.git

Load data in train_file and test_file.
Add a column of language.

In [ ]:
xquad_train_file = '/kaggle/working/xquad/xquad.hi.json'

with open(xquad_train_file, 'r') as file_input:
    train_file = json.load(file_input)
    
examples_train = preprocess(train_file, 'dev')
xquad = pd.DataFrame(examples_train)
xquad['language'] = 'hindi'

Delete extra file and folder

In [ ]:
import os, shutil
folder = '/kaggle/working/'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

In [ ]:
mlqa.to_csv('mlqa_hindi.csv', index=False)
xquad.to_csv('xquad.csv', index=False)

# Pls do UPVOTE